# PDF Preprocessing and Documents Creations

---



Nowadays one of the challenging aspects of preprocessing scientific documents, for the implementation of a RAG (*Retrieval-Augmented Generation*) system, is the accurate **extraction of mathematical formulas**. Considering working with files containing this type of information, we faced the need to adopt a solution that would allow us to achieve results as good and reliable as possible using free tools and resources.

In addition to this, for completeness we report other main relevant tasks, regarding the processing of files in order to implement a RAG system, we tried to manage:
*   **handle complex structures** mantaning contextual meaning;
*   **text cleaning** removing irrilevant elements;
*   **chunk size balancing** accordingly with the specific context limit for the chosen LLM;
*   **text splitting** avoiding breaking meaningful connections in sentences.

For this purpose we propose an hybrid approach that combines multiple solutions. In particular we built a pipeline that integrates functionalities of two solutions:
*   `Unstructure`, an open-source tool efficient in transforming unstructured data into structured outputs.
    *   https://github.com/Unstructured-IO
*   `Nougat`, an advanced Transformer-based OCR model that simplifies the process of converting complex scientific contents into a common and machine-readable format.
    *   https://github.com/facebookresearch/nougat

The tasks we have addressed to these tools are respectively:
1.   Parse the PDF file obtaning **elements characterized by type and content** through `Unstructure partition function`. This allowed us to:
    *   clean the extracted contents **filering out irrelevant or meaningless types** of elements;
    *   identify page breaks, permetting to implement a **page content-aware solution**;
    *   **locate the formulas**, saving indexes of pages containing them;
    *   translate elements into **a dataframe, with columns Type and Value**, for easier handling.

2.   Extract formulas in more accurate and replicable Latex format thanks to `Nougat`. This was done according to the following logic:
    *   **only pages in** which we know **there is at least one formula are processed**, allowing us to save resources and time by avoiding analyzing the pages of the file in which no formulas appear;
    *   the selected pages are converted into images, analyzed using a **Transformer-based OCR model**, and returned as text formatted in LaTeX;
    *   the model's output is analyzed to identify **Latex patterns matching formulas**, which are then extracted.

This approach was necessary because, despite employing a high-resolution element identification strategy in Unstructured partitioning, the formulas often turned out incorrect and failed to meet our expectations. Therefore, we retained the use of `Unstructured` for its ability to implement an **elements and page-aware solution** while integrating `Nougat` in a specialized manner **for extracting mathematical formulas**. This combination allowed us to achieve significantly improved and optimal results.

Continuing with the numbering used above, the final processing steps are:
3.   **Replace the more precise formulas in the dataframe** where all the elements are stored. By doing this we faced the problem of mismatch between expected and extracted formulas for a single page:
    *   if no formulas extracted by `Nougat` we keep the formulas as extracted by `Unstructure`;
    *   else, if the number of extracted formulas differs from the expected one, we give priority to the formulas extracted by `Nougat`, due to more precision and accuracy, discarding the ones extracted by `Unstructure`.
4.   **Page-based Chunks creation**. This was done according to the following logic:
    *   setting a **threshold as the maximum number of tokens** that can be included into a single chunk;
    *   creating chunks by **concatenating page-by-page content** to preserve contextual integrity as much as possible;
    *   if the content of a single page exceeds the threshold, we concatenate **elements one by one until the content remains under the limit**. This approach is a tradeoff between the balancing of chunk sizes and the coherence of contextual structures.

Let's dive into the implementation.


Installing required libraries.

In [1]:
#!pip install -q torch gdown huggingface_hub python-dotenv transformers sentence-transformers pymupdf python-Levenshtein nltk chromadb tqdm unidecode gradio bitsandbytes seaborn unstructured[pdf] langchain langchain-community langchain-huggingface latex2mathml --progress-bar off

In [2]:
# su windows serve, non sono sicuro su Colab
#!pip install -q python-magic-bin==0.4.14

In [3]:
# on windows
# !conda install -c conda-forge poppler -y
# !conda install -c conda-forge tesseract -y

# on Colab/Unix
# !apt-get install poppler-utils tesseract-ocr

In [ ]:
# import os
# import sys

# # Determina dinamicamente il percorso dell'ambiente Conda
# conda_env_path = os.path.dirname(sys.executable)  # Ottiene il percorso dell'eseguibile Python
# tessdata_path = os.path.join(conda_env_path, "share", "tessdata")  # Percorso a tessdata

# # Imposta la variabile di ambiente TESSDATA_PREFIX
# os.environ["TESSDATA_PREFIX"] = os.path.join(conda_env_path, "share")
# print("TESSDATA_PREFIX:", os.environ["TESSDATA_PREFIX"])

# # Verifica se il percorso è corretto
# if os.path.exists(tessdata_path):
#     print(f"Tessdata directory found: {tessdata_path}")
# else:
#     print(f"Tessdata directory not found: {tessdata_path}")

In [2]:
import os
import torch
from huggingface_hub import login
from dotenv import load_dotenv
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import time
from tqdm import tqdm
import re
import gradio as gr
from unidecode import unidecode
import random
import nltk
import gdown

In [ ]:
print(nltk.__version__)  # confirm you see 3.9.1, otherwise restart session

nltk.download("punkt")

## Environment Setup
Here we detect if this notebook is being executed in a **Google Colab environment** or a **local system**, setting up the environment, loading keys accordingly and downloading required files.



> Note that, in order to speed up the execution, we download also the `elements.pkl` file to avoid some expensive processing (*about 30 minute*).



In [ ]:
PDF_NAME = "Information Retrieval Slides.pdf"
running_on_colab = False

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("Running in Google Colab. Using userdata to get HF_TOKEN.")
    running_on_colab = True

except ModuleNotFoundError:
    load_dotenv()
    HF_TOKEN = os.getenv('HF_TOKEN')
    print("Not running in Google Colab. Using load_dotenv to get HF_TOKEN.")

file_path = PDF_NAME
if os.path.exists(file_path):
    print(f"File {file_path} already exists.")
else:
    # save locally from https://drive.google.com/file/d/1xUA6_ZBJzWGF7kWpM1YZTK3R1siYg1qY/view?usp=drive_link
    gdown.download(id="1xUA6_ZBJzWGF7kWpM1YZTK3R1siYg1qY", output=PDF_NAME, quiet=False)

    # download of pickled elements to speed up the execution
    gdown.download(id="17eXYgmiTL9-f9F5vIT5j7QffHIu_QwuN", output="elements.pkl", quiet=False)

# Logs into Hugging Face using the token
login(token=HF_TOKEN)

## PDF Partitioning with Unstructured
Processes the PDF using the `partition` function to extract structured elements.

We decide to apply the **high-resolution strategy** because is highly sensitive, it is recommended if you want obtain precise classifications for document elements. Adopting this strategy we are able to use the document layout to gain additional information about document elements, in particular we are able to recognize:
- **Titles** and **Text** (we use them as it is)
- **Formulas** (we further process them)
- **Images** (will be removed)
- **Page Breaks** (fundamental to handle formulas and create chuncks)


In [ ]:
from unstructured.partition.auto import partition
from tqdm import tqdm
import fitz  # PyMuPDF
import pickle

file_name = "Information Retrieval Slides.pdf"
title = "Information Retrieval"

load_elements_from_pickle = True
elements = []

if load_elements_from_pickle:
    # Loading the pickled elements.
    with open('elements.pkl', 'rb') as f:
        elements = pickle.load(f)
else:
    # Apri il PDF
    doc = fitz.open(file_name)  # Apri il PDF con PyMuPDF
    total_pages = len(doc)
    
    for page_number in tqdm(range(total_pages), desc="Processing PDF pages"):
        # Crea un nuovo documento con una sola pagina
        temp_doc = fitz.open()  # Documento vuoto
        temp_doc.insert_pdf(doc, from_page=page_number, to_page=page_number)
        
        # Salva la singola pagina in un file temporaneo
        temp_page_file = f"temp_page_{page_number}.pdf"
        temp_doc.save(temp_page_file)
        temp_doc.close()
        
        # Applica `partition` sulla singola pagina
        page_elements = partition(
            filename=temp_page_file,
            strategy="hi_res",
            skip_infer_table_types=[],
            include_page_breaks=True,
        )
        elements.extend(page_elements)  # Aggiungi gli elementi alla lista totale
    
    doc.close()  # Chiudi il documento principale

# Stampa il numero totale di elementi
print("Number of recognized elements: " + str(len(elements)) + " in file " + file_name)


Storing the elements in a pickle file.
Snippet of code executed if needed.

In [6]:
# to uncomment, now is commented cause we are downloading the pickled elements
with open('elements.pkl', 'wb') as f:
    pickle.dump(elements, f)

## Filtering and Analyzing Extracted Elements

Here we build a **dataframe**, creating pairs of **Type** and **Value**, using the elements extracted from the PDF file.

Then we print all the different element's Type recognized.



In [ ]:
type_value_elements = []

for i in range(len(elements)):
  pair = [type(elements[i]), elements[i].text]
  type_value_elements.append(pair)

import pandas as pd
df_elements = pd.DataFrame(type_value_elements, columns=['Type', 'Value'])

# print all the different possible Type in the dataframe
print(df_elements['Type'].unique())

Now we clean the dataframe filtering out all the useless elements:
- **Images** (for them we have a text extracted with OCR technique, but not very useful for the language model due to a loss of meaning)
- **Email Addresses**
- **Table** (similarly to the images they lost some meaning as they are extracted)
- **Footers** and **Headers**


In [ ]:
from unstructured.documents.elements import Image, EmailAddress, FigureCaption, Table, Footer, Header, PageBreak

# print df number of rows
print("Number of actual elements: " + str(len(df_elements)))

# drop all the rows with unuseful Types
df_elements = df_elements[df_elements['Type'] != Image]
#df_elements = df_elements[df_elements['Type'] != EmailAddress]
df_elements = df_elements[df_elements['Type'] != Table]
df_elements = df_elements[df_elements['Type'] != Footer]
df_elements = df_elements[df_elements['Type'] != Header]
df_elements = df_elements.drop(df_elements[(df_elements['Type'] != PageBreak) & (df_elements['Value'] == '')].index)

# print df number of rows
print("Number of elements after filtering: " + str(len(df_elements)))

# reassign the dataframe index
df_elements = df_elements.reset_index(drop=True)

# print all the different possible Type in the dataframe
print(df_elements['Type'].unique())


Check if there are some empty-valued elements, that are not Page Breaks, in the dataframe.

In [ ]:
# print rows of df_elements with bith Type != PageBreak and Value empty
print(len(df_elements[(df_elements['Type'] != PageBreak) & (df_elements['Value'] == '')]))

if len(df_elements[(df_elements['Type'] != PageBreak) & (df_elements['Value'] == '')]) > 0:
  df_elements = df_elements.drop(df_elements[(df_elements['Type'] != PageBreak) & (df_elements['Value'] == '')].index)

## Formulas Detection
Here we identify and track pages containing formulas in the PDF.

- `page_with_formula_indexes`: stores page indexes with formulas.
- `number_of_formulas_per_page`: tracks the count of formulas per page.
- `locations_with_formula_at_page`: stores locations of formulas in the DataFrame by page.


In [ ]:
from unstructured.documents.elements import Formula, PageBreak

page_with_formula_indexes = []

# dictionary to store the number of formula for each page
number_of_formulas_per_page = {}
locations_with_formula_at_page = {}

index = 0  # first page -> index = 0

# iterate over df_elements
for i in range(len(df_elements)):
  if df_elements.iloc[i]['Type'] == PageBreak:
    index += 1
  if df_elements.iloc[i]['Type'] == Formula: #or df_elements.iloc[i]['Type'] == Image:
    # saving in dictionary the dataframe locations with formula/image for that page
    if index not in locations_with_formula_at_page:
      locations_with_formula_at_page[index] = [i]
    else:
      locations_with_formula_at_page[index].append(i)
    # saving in dictionary the number of formula/image for that page
    if index not in number_of_formulas_per_page:
      number_of_formulas_per_page[index] = 1
    else:
      number_of_formulas_per_page[index] += 1
    if index not in page_with_formula_indexes:
      page_with_formula_indexes.append(index)

print("Pages with formulas indexes")
print(page_with_formula_indexes)
print("Number of pages with at least one formula: " + str(len(page_with_formula_indexes)))

## Loading Nougat Model and Processor
This code initializes the `Nougat` model and its processor for handling vision-to-text tasks.


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
%%capture
from transformers import AutoProcessor, VisionEncoderDecoderModel

# Load the Nougat model and processor from the hub
processor = AutoProcessor.from_pretrained("facebook/nougat-small")
model = VisionEncoderDecoderModel.from_pretrained("facebook/nougat-small").to(device)

The following block of code is taken from the example at this link from `Nougat` GitHub: https://github.com/inuwamobarak/nougat/blob/main/Nougat_Inference_to_read_scientific_PDFs.ipynb

In [13]:
from typing import Optional, List
import io
import fitz
from pathlib import Path

def rasterize_paper(
    pdf: Path,
    outpath: Optional[Path] = None,
    dpi: int = 96,
    return_pil=False,
    pages=None,
) -> Optional[List[io.BytesIO]]:
    """
    Rasterize a PDF file to PNG images.

    Args:
        pdf (Path): The path to the PDF file.
        outpath (Optional[Path], optional): The output directory. If None, the PIL images will be returned instead. Defaults to None.
        dpi (int, optional): The output DPI. Defaults to 96.
        return_pil (bool, optional): Whether to return the PIL images instead of writing them to disk. Defaults to False.
        pages (Optional[List[int]], optional): The pages to rasterize. If None, all pages will be rasterized. Defaults to None.

    Returns:
        Optional[List[io.BytesIO]]: The PIL images if `return_pil` is True, otherwise None.
    """

    pillow_images = []
    if outpath is None:
        return_pil = True
    try:
        if isinstance(pdf, (str, Path)):
            pdf = fitz.open(pdf)
        if pages is None:
            pages = range(len(pdf))
        for i in pages:
            page_bytes: bytes = pdf[i].get_pixmap(dpi=dpi).pil_tobytes(format="PNG")
            if return_pil:
                pillow_images.append(io.BytesIO(page_bytes))
            else:
                with (outpath / ("%02d.png" % (i + 1))).open("wb") as f:
                    f.write(page_bytes)
    except Exception:
        pass
    if return_pil:
        return pillow_images

## Rasterizing PDF Pages for Formula Processing
Now we extract and process specific pages containing formulas from a PDF.

In particular the processing is made in order to obtain the rasterized **images of specific pages** of the PDF file.


In [ ]:
from huggingface_hub import hf_hub_download
from typing import Optional, List
import io
import fitz
from pathlib import Path
from PIL import Image

#file_name = "Information Retrieval Slides"

images = rasterize_paper(pdf=file_name, return_pil=True, pages=page_with_formula_indexes)

print("Number of processed pages: " + str(len(images)))

# Display the first image
image = Image.open(images[0])
image


The following block of code is taken from the example at this link from `Nougat` GitHub: https://github.com/inuwamobarak/nougat/blob/main/Nougat_Inference_to_read_scientific_PDFs.ipynb

In [15]:
from transformers import StoppingCriteria, StoppingCriteriaList
from collections import defaultdict

class RunningVarTorch:
    def __init__(self, L=15, norm=False):
        self.values = None
        self.L = L
        self.norm = norm

    def push(self, x: torch.Tensor):
        assert x.dim() == 1
        if self.values is None:
            self.values = x[:, None]
        elif self.values.shape[1] < self.L:
            self.values = torch.cat((self.values, x[:, None]), 1)
        else:
            self.values = torch.cat((self.values[:, 1:], x[:, None]), 1)

    def variance(self):
        if self.values is None:
            return
        if self.norm:
            return torch.var(self.values, 1) / self.values.shape[1]
        else:
            return torch.var(self.values, 1)


class StoppingCriteriaScores(StoppingCriteria):
    def __init__(self, threshold: float = 0.015, window_size: int = 200):
        super().__init__()
        self.threshold = threshold
        self.vars = RunningVarTorch(norm=True)
        self.varvars = RunningVarTorch(L=window_size)
        self.stop_inds = defaultdict(int)
        self.stopped = defaultdict(bool)
        self.size = 0
        self.window_size = window_size

    @torch.no_grad()
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        last_scores = scores[-1]
        self.vars.push(last_scores.max(1)[0].float().cpu())
        self.varvars.push(self.vars.variance())
        self.size += 1
        if self.size < self.window_size:
            return False

        varvar = self.varvars.variance()
        for b in range(len(last_scores)):
            if varvar[b] < self.threshold:
                if self.stop_inds[b] > 0 and not self.stopped[b]:
                    self.stopped[b] = self.stop_inds[b] >= self.size
                else:
                    self.stop_inds[b] = int(
                        min(max(self.size, 1) * 1.15 + 150 + self.window_size, 4095)
                    )
            else:
                self.stop_inds[b] = 0
                self.stopped[b] = False
        return all(self.stopped.values()) and len(self.stopped) > 0

## Generating Outputs from Processed Images
Once we obtained an image for each page containing at least a formula, we can generate text outputs from images using the `Nougat` model.

- `outputs` is the list to store generated text for each image
- The **image processing** converts images into tensor format using the `processor`.
- For the **text generation** the tensor is passed to the model's `generate` method.

Note that this block require a compatible GPU, for efficient processing, indicated by `device`.



In [ ]:
outputs = []

for i in range(len(images)):

  image = Image.open(images[i])

  pixel_values = processor(images=image, return_tensors="pt").pixel_values

  outputs.append(
        model.generate(
        pixel_values.to(device),
        min_length=1,
        max_length=3584,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
        output_scores=True,
        stopping_criteria=StoppingCriteriaList([StoppingCriteriaScores()]),
    )
  )

This function extracts LaTeX formulas enclosed within `\[ \]` from a given text.

In [ ]:
def formulas_extraction(latex_text):
    """
    Extracts all substrings that start with '\[' and end with '\]' and returns them as a list.
    """
    # The pattern looks for:
    #   - literal "\["
    #   - any characters, non-greedily (.*?)
    #   - literal "\]"
    pattern = r"(\\\[.*?\\\])"

    matches = re.findall(pattern, latex_text)

    return matches

# Post-Processing Generated Formulas
Now we are able to process the generated LaTeX formulas for each page and update the corresponding DataFrame entries.

- **Purpose:**
  - Ensures that the number of extracted formulas matches the expected count per page.
  - Handles cases where formulas are fewer or greater than expected.

- **Workflow:**
  1. **Formula Extraction:**
     - Decodes the generated text and extracts formulas using `formulas_extraction`.
  2. **Case Handling:**
     - **No extracted Formulas:** if no formulas extracted by `Nougat` we can keep the formulas as extracted by `Unstructure`.
     - **Fewer extracted Formulas:** concatenate formulas extracted by `Nougat` into a single string and update the first location in the DataFrame, marking others as `REMOVING_STR`.
     - **More extracted Formulas:** concatenate and store in the first location, similar to fewer formulas.
     - **Matching Formulas:** update the DataFrame with extracted formulas directly.

Note that we give priority to `Nougat` formulas, if extracred, due to a more precise and correct representation.


In [ ]:
from tkinter.constants import E
from unstructured.documents.elements import Title, NarrativeText, Image, Formula, PageBreak

cases_ok = 0
cases_not_improved = 0
cases_under_expected = 0
cases_over_expected = 0

old_dataframe = df_elements.copy()
old_formulas = []
new_formulas = []

REMOVING_STR = "LOC_TO_BE_REMOVED"

for i in range(len(outputs)):

  page_index = page_with_formula_indexes[i]

  generated = processor.batch_decode(outputs[i][0], skip_special_tokens=True)[0]
  generated = processor.post_process_generation(generated, fix_markdown=False)

  formulas = formulas_extraction(generated)

  if len(formulas) == 0:
    cases_not_improved += 1
    continue

  num_over_expected = number_of_formulas_per_page[page_index] - len(formulas)

  if num_over_expected < 0:

    cases_under_expected += 1

    # concatenate all elements in formulas in a unique string
    concatenated_formulas = " ".join(formulas)

    for j in range(number_of_formulas_per_page[page_index]):
      if j == 0:
        df_elements.at[locations_with_formula_at_page[page_index][j], 'Value'] = concatenated_formulas
      else:
        df_elements.at[locations_with_formula_at_page[page_index][j], 'Value'] = REMOVING_STR

  elif num_over_expected > 0:

    cases_over_expected += 1

    # concatenate all elements in formulas in a unique string
    concatenated_formulas = " ".join(formulas)

    for j in range(number_of_formulas_per_page[page_index]):
      if j == 0:
        df_elements.at[locations_with_formula_at_page[page_index][j], 'Value'] = concatenated_formulas
      else:
        df_elements.at[locations_with_formula_at_page[page_index][j], 'Value'] = REMOVING_STR

  else:
    cases_ok += 1
    for j in range(number_of_formulas_per_page[page_index]):
      df_elements.at[locations_with_formula_at_page[page_index][j], 'Value'] = formulas[j]
      old_formulas.append(formulas[j])
      new_formulas.append(df_elements.at[locations_with_formula_at_page[page_index][j], 'Value'])

print("Cases ok: " + str(cases_ok))
print("Cases not improved: " + str(cases_not_improved))
print("Cases under expected: " + str(cases_under_expected))
print("Cases over expected: " + str(cases_over_expected))

In [ ]:
# print df row with value REMOVING_STR
print(df_elements[df_elements['Value'] == REMOVING_STR])

In [ ]:
# drop df row with value REMOVING_STR
df_elements = df_elements[df_elements['Value'] != REMOVING_STR]

# reset index
df_elements = df_elements.reset_index(drop=True)

print(df_elements[df_elements['Value'] == REMOVING_STR])

In [ ]:
from unstructured.documents.elements import Formula

# print df_elements for rows with Type Formula
print(df_elements[df_elements['Type'].isin([Formula])])

Here we clean up the `df_elements` DataFrame by removing consecutive duplicate rows while keeping the first occurrence.


In [ ]:
print("Number of elements before removing adjacent duplicates: " + str(len(df_elements)))

# drop adjacent duplicates in the df_elements mantaining the first one
df_elements = df_elements.loc[df_elements.shift().ne(df_elements).any(axis=1)]

# reset index
df_elements = df_elements.reset_index(drop=True)

print("Number of elements after removing adjacent duplicates: " + str(len(df_elements)))

In [ ]:
from unstructured.documents.elements import PageBreak

# count the number of rows with Type PageBreak
print(len(df_elements[df_elements['Type'] == PageBreak]))

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')

def count_tokens(text):
  return len(word_tokenize(text))

Here we compute some statistics regarding the number of tokens in the pages.

In [ ]:
# compute the max, the min and the average number of tokens in a page
# by concatenating elements of the dataframe between consecutive page breaks

num_token_per_page = []

current_page = ""
current_page_tokens = 0

for i in range(len(df_elements)):
  if df_elements.iloc[i]['Type'] != PageBreak:
    current_page += df_elements.iloc[i]['Value']
    current_page_tokens = count_tokens(current_page)
    #current_page_tokens += count_tokens(df_elements.iloc[i]['Value'])
  else:
    num_token_per_page.append(current_page_tokens)
    current_page = ""
    current_page_tokens = 0

print("Max tokens: " + str(max(num_token_per_page)))
print("Min tokens: " + str(min(num_token_per_page)))
print("Avg tokens: " + str(sum(num_token_per_page)/len(num_token_per_page)))

In [ ]:
# plot the distribution of number of tokens per page showing the mean
import matplotlib.pyplot as plt
import numpy as np

# Calculate mean
mean_tokens = np.mean(num_token_per_page)

# Create the plot
plt.figure(figsize=(9, 5))
plt.hist(num_token_per_page, bins=10, edgecolor='black', alpha=0.5)
plt.axvline(mean_tokens, color='red', linestyle='dashed', linewidth=1, label=f'Mean = {mean_tokens:.2f}')
plt.title('Distribution of Number of Tokens per Page')
plt.xlabel('Number of Tokens')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()


# Splitting Content into Tokenized Chunks
Here we divide the processed elements in `df_elements` into smaller text files, ensuring each file stays within a token limit.

**Chunk Creation:**
- Processes elements in `df_elements`:
  - Concatenates values until the token count exceeds `doc_tokens_threshold`.
  - Writes each chunk to a separate text file (`docX.txt`).
  - Tracks the token count of each chunk in `docs_len`.
- Handles `PageBreak` to reset and manage document boundaries.

Adjust `doc_tokens_threshold` to change the maximum token count per file.


In [ ]:
import os

# create folder chunks if it doesn't exist
if not os.path.exists('chunks'):
    os.makedirs('chunks')
else:
    # delete all the files in the folder chunks
    for file in os.listdir('chunks'):
        os.remove(os.path.join('chunks', file))


block = ""
document = ""
doc_index = 0
doc_tokens_threshold = 150
docs_len = []

# iterating over df_elements
for i in range(len(df_elements)):


  if df_elements.at[i, 'Type'] != PageBreak:

    next_element = df_elements.at[i, 'Value'] + " "

    if count_tokens(block) + count_tokens(next_element) > doc_tokens_threshold:
        with open('chunks/document_' + str(doc_index) + '.txt', 'w') as f:
          f.write(block)

        docs_len.append(count_tokens(block))
        doc_index += 1
        block = ""

    block += next_element

  else: # if the type is a PageBreak

    expected_num_tokens = count_tokens(document) + count_tokens(block)
    if expected_num_tokens > doc_tokens_threshold:

      with open('chunks/document_' + str(doc_index) + '.txt', 'w') as f:
        f.write(document)

      docs_len.append(count_tokens(document))
      doc_index += 1
      document = ""

    document += block
    block = ""

print("Number of chunks obtained: " + str(doc_index))

In [ ]:
# print the max, the min and the avg of values in docs_len
print(len(docs_len))

print(max(docs_len))
print(min(docs_len))
print(sum(docs_len)/len(docs_len))

In [ ]:
# plot the distribution of number of tokens per document showing the mean
import matplotlib.pyplot as plt
import numpy as np

# Calculate mean
mean_tokens = np.mean(docs_len)

# Create the plot
plt.figure(figsize=(9, 5))
plt.hist(docs_len, bins=10, edgecolor='black', alpha=0.5)
plt.axvline(mean_tokens, color='red', linestyle='dashed', linewidth=1, label=f'Mean = {mean_tokens:.2f}')
plt.title('Distribution of Number of Tokens per Document')
plt.xlabel('Number of Tokens')
plt.ylabel('Frequency')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.show()

In [ ]:
# Spostiamo il modello su CPU
model.to("cpu")

# Rimuoviamo il modello dalla memoria
del model

# Libera la memoria non più necessaria dalla GPU
torch.cuda.empty_cache()

# FROM HERE THE LM NOTEBOOK

# RAG Pipeline on PDFs with Limited GPU Requirements

In this project, we aim to build a **Retrieval-Augmented Generation (RAG)** pipeline optimized for **limited GPU environments**. The goal is to combine the power of *dense retrieval-based methods* with the flexibility of generative models, while ensuring the system remains efficient enough to run on medium-tier laptops with GPUs.

A RAG pipeline consists of two main stages: **retrieval** and **generation**. The entire process can be divided into **3 key steps**:

1. **Generation of document embeddings**  
   In this step, document embeddings are generated from a *corpus* using an embedding model. Each document is encoded into a dense vector representation, capturing semantic information.

2. **Document retrieval**  
   Relevant documents are fetched from the corpus based on the input query. This is achieved by utilizing the generated embeddings and calculating similarity with the query's embedding to retrieve the most relevant documents.

3. **Output generation**  
   The retrieved documents are then passed to a **generative model**, which produces contextually relevant responses based on the information extracted from the documents.

By leveraging lightweight models and optimizing for efficiency, this RAG pipeline delivers powerful **AI-driven results** even on hardware with **limited resources**.

Finally, a **Gradio interface** is provided, allowing users to interact with the system and "chat" with the documents they provide, offering a seamless experience for exploring the content.

## Hardware requirements and constraints

Let's find out what hardware we've got available to see what kind of model(s) we'll be able to load.

> **Note:** You can also check this with the `!nvidia-smi` command.

In [ ]:
# Get GPU available memory
gpu_memory_bytes = torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb = round(gpu_memory_bytes / (2**30))
print(f"Available GPU memory: {gpu_memory_gb} GB")
print(torch.cuda.get_device_name(0))

Of course, depending on the provided harware, better models can be utilized.

### Checking local GPU memory availability

Let's first analyze how we decided the model to use for this project. This notebook was primarily run and tested locally on a **laptop** with 16GB of RAM and an NVIDIA RTX 3070 laptop GPU (8GB of VRAM). The main goal was to create a pipeline that could efficiently run on this portable device, leveraging the benefits of GPUs and CUDA for AI tasks while ensuring that the performance and capabilities did not feel lacking compared to larger models.

We will need two main ingredients:
- An *embedder* model, that calculates dense embeddings from documents
- An *LLM*, that provides output given user's queries and the retrieved documents

In their dedicated sections we will uncover the choiches made for both.

### Load our PDF and start producing chunks

Let's now start by loading a pdf file and extracting chunks from it. These chunks' **quality** is important, since these will be essentially the *documents* on which we will compute the embeddings on.

In [ ]:
PDF_NAME = "Information Retrieval Slides.pdf"
running_on_colab = False

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("Running in Google Colab. Using userdata to get HF_TOKEN.")
    running_on_colab = True
except ModuleNotFoundError:
    load_dotenv()
    HF_TOKEN = os.getenv('HF_TOKEN')
    print("Not running in Google Colab. Using load_dotenv to get HF_TOKEN.")

file_path = PDF_NAME
login(token=HF_TOKEN)

In [34]:
def preprocess_text(text):
    # Rimuovi formule matematiche LaTeX
    text = re.sub(r'\$.*?\$', '', text)
    # Rimuovi caratteri non alfanumerici eccetto punteggiatura di base
    text = re.sub(r'[^a-zA-Z0-9\s.,;:!?\'"-]', '', text)
    # Rimuovi •
    text = text.replace('•', '')
    # Normalizza i caratteri Unicode
    text = unidecode(text)
    # Rimuovi spazi multipli
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def aggregate_short_documents(texts, min_length=50):
    aggregated_texts = []
    buffer = ""

    for doc in texts:
        if len(doc.page_content) < min_length:
            buffer += " " + doc.page_content
            buffer = buffer.strip()
        else:
            if buffer:
                doc.page_content = buffer + " " + doc.page_content
                buffer = ""
            aggregated_texts.append(doc)

    if buffer:
        if aggregated_texts:
            aggregated_texts[-1].page_content += " " + buffer
        else:
            aggregated_texts.append(buffer)

    return aggregated_texts

# ------------------------------------------------------------------------- #

# Load and process the PDF
loader = PyMuPDFLoader(file_path=file_path)
documents = loader.load()

# Split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separator="\n")
texts = text_splitter.split_documents(documents)

# Preprocess the text
for doc in texts:
    doc.page_content = preprocess_text(doc.page_content)

# Aggrega i documenti corti
corpus = aggregate_short_documents(texts)

In [ ]:
# HERE NEW CODE

corpus = []
# for each document in chunks folder append its text to corpus
for file in sorted(os.listdir("chunks")):
    with open(os.path.join("chunks", file), "r", encoding="utf-8") as f:
        corpus.append(f.read())

print(len(corpus))

Let's give a look to some random documents extracted and preprocessed from the initial pdf

In [ ]:
# print some random pages with their indices
random.seed(42)
for i in range(5):
    doc = random.choice(corpus)
    doc_index = corpus.index(doc)
    #print(f"DocId: {doc_index}, {doc.page_content[:100]}...")
    print(f"DocId: {doc_index}, {doc[:100]}...")

In [ ]:
#print(f"Average document length: {sum(len(doc.page_content) for doc in corpus) / len(corpus)}")
print(f"Average document length: {sum(len(doc) for doc in corpus) / len(corpus)}")

We also provide a `save_documents` function, which can be useful for further inspection of the PDF content.

In [ ]:
'''import os

def save_documents(documents):
    """
    Save the provided documents to the vector_store directory. Each document is saved as a separate text file.
    """
    if not os.path.exists("vector_store"):
        os.makedirs("vector_store")
    # delete all files in the vector_store directory
    for file in os.listdir("vector_store"):
        os.remove(os.path.join("vector_store", file))
    for i, doc in enumerate(documents):
        with open(f"vector_store/document_{i}.txt", "w", encoding="utf-8") as f:
            f.write(doc.page_content)'''

In [ ]:
#save_documents(corpus)
print(f"Saved {len(corpus)} documents in the vector_store directory.")

In [ ]:
import json

def extract_questions(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    questions = []
    for item in data:
        # q = item.get("question", "")
        # a = item.get("answers", "")
        # # 'a' è una lista di stringhe, aggiungi ad ognuno 1, 2, 3, 4
        # for i in range(len(a)):
        #     a[i] = f"{(1 + i)}) {a[i]}"
        # a = " ".join(a)
        # qplusa = f"{q} {a}"
        if str(item.get("category", "")) == "1":
            continue
        question = {
            "question": item.get("question", "").strip(),
            # "answers": item.get("answers", ""),
            # "correct": item.get("correct", ""),
            # "category": item.get("category", ""),
            # "question_id": item.get("question_id", "")
        }
        questions.append(question)
    return questions


file_path = "quiz/quiz.json"

if running_on_colab:
  if not os.path.exists("quiz"):
      os.makedirs("quiz")
  # save locally from https://drive.google.com/file/d/15pKNg7XwtaUILmxUhzhNa-ojTpqqLg__/view?usp=drive_link
  gdown.download(id="15pKNg7XwtaUILmxUhzhNa-ojTpqqLg__", output=file_path, quiet=False)

queries = extract_questions(file_path)
print(f"Loaded {len(queries)} questions.")

random.seed(4242)
print(random.choice(queries))

## Gather Dataset Embeddings

In this first stage, the aim is to produce the documents embeddings, in order to be able to make similarity searches in the upcoming steps of the pipeline.

### About the `cde-small-v1` Model

The `cde-small-v1` model, developed by John X. Morris and Alexander M. Rush, is a cutting-edge model for generating **Contextual Document Embeddings (CDE)** ([link](https://huggingface.co/jxm/cde-small-v1)). What sets this model apart is its ability to integrate "*context tokens*" into the embedding process, which allows it to capture the nuances and relationships between documents more effectively. This makes it particularly suitable for generating highly accurate embeddings for both documents and queries, especially in cases where capturing the context of a document within the broader corpus is crucial.

We chose this model because, as of December 2024, it is one of the leading models under 400M parameters, delivering impressive results on the [**Massive Text Embedding Benchmark (MTEB) leaderboard**](https://huggingface.co/spaces/mteb/leaderboard). Although it ranks 32nd overall, it stands out as the top model in terms of **memory efficiency**, which is a key factor for our project, given the requirement for **limited GPU capabilities**. Additionally, it offers a substantial **embedding dimension of 768**, striking a balance between computational efficiency and embedding quality.

<style>
    img {
        border-radius: 15px;
    }
</style>
![cde-small-v1.png](https://github.com/giuliocapecchi/LM-project/blob/main/assets/cde-small-v1.png?raw=1)

A key feature of the `cde-small-v1` model is its optimization for a two-stage embedding process:

1. **First Stage**: Embedding a subset of documents from the corpus to create "dataset embeddings," which serve as a **reference** for the entire corpus.
2. **Second Stage**: Using the dataset embeddings to embed new queries and documents during inference.

This model is compact yet delivers solid performance, making it suitable for our use case.

### Steps to Gather Dataset Embeddings:

1. **Selecting a Subset of Documents**  
   We begin by sampling a representative set of documents from the corpus. Following the model's guidelines, we select 512 documents. If this number isn't available, the model can handle oversampling, which is the case for the PDF dataset we're using. Despite this, performance remains strong.

2. **Generating Dataset Embeddings**  
   After selecting the documents, we encode them using the `cde-small-v1` embedding model. This step produces dense vector representations of the documents, which are representative of the broader corpus.

3. **Embedding Queries and Documents**  
   Once the dataset embeddings are created, we use them to embed both documents and queries. A key feature of this model is its ability to differentiate between 'queries' and 'documents' during encoding, ensuring context is preserved during the embedding process.


Next, we will load the model using the `SentenceTransformers` interface to begin the embedding process.

In [41]:
%%capture

from sentence_transformers import SentenceTransformer

# with sentence-transformers we don't need prefixes but, to do retrieval, we need to use prompt_name="query" and prompt_name="document" in the encode method of the model when embedding queries and documents, respectively.
embeddings_model = SentenceTransformer(
    "jxm/cde-small-v1",
    trust_remote_code=True,
).to(device)

During the development of the project, the creators of the embedding model made a change in the Hugging Face repository, which led to unstable results when using the `SentenceTransformer` implementation. If a similar issue arises in the future, it can be mitigated by specifying the `revision` and `tokenizer_kwargs` parameters in the model constructor. These parameters allow to lock the model and tokenizer to a specific branch name, tag, or commit ID from the Hugging Face repository, ensuring stability.

Here’s how to implement it:

```python
embeddings_model = SentenceTransformer(
    "jxm/cde-small-v1",
    trust_remote_code=True,
    revision="9e2ed1d8d569d34458913d2d246935c1b2324d11",  # Latest stable model revision
    tokenizer_kwargs={"revision": "86b5e0934494bd15c9632b12f734a8a67f723594"}  # Latest stable tokenizer revision
).to(device)
```

The tags provided above correspond to the latest stable commits (as of December 2024). You can retrieve them directly from the model's card page on Hugging Face, under the "Files and Versions" tab.

Fortunately for us, the issue was resolved the same day we reported it to the creators via Twitter. We thank them for their prompt response and the fix they provided.

Let's follow up by producing the `minicorpus` (which is the subsample of the whole corpus) and what we called `dataset_embeddings`

In [ ]:
minicorpus_size = embeddings_model[0].config.transductive_corpus_size # 512
random.seed(4242)
minicorpus_docs = random.choices(corpus, k=minicorpus_size) # oversampling is okay
assert len(minicorpus_docs) == minicorpus_size # We must use exactly this many documents in the minicorpus

dataset_embeddings = embeddings_model.encode(
    #[doc.page_content for doc in minicorpus_docs],
    [doc for doc in minicorpus_docs],
    prompt_name="document",
    convert_to_tensor=True,
    show_progress_bar=True
)

print(f"Corpus size: {len(corpus)}")
print(f"Computed embeddings for {len(minicorpus_docs)} documents. Shape: {dataset_embeddings.shape}")

In [ ]:
print("Some mini-corpus documents:")

# get some random documents from the minicorpus
random.seed(4242)
for i in random.sample(range(minicorpus_size), 5):
    #print(f"document {i}: {minicorpus_docs[i].page_content[:100]}...")
    print(f"document {i}: {minicorpus_docs[i][:100]}...")

Now that we have obtained the **dataset embeddings**, we can proceed to embed both documents and queries using the same model.

To embed the documents and queries, we must ensure that we specify the correct `prompt_name` for each, as well as pass the `dataset_embeddings` to maintain context. The reason is that, as many state-of-the-art-models, this one was trained with task-specific prefixes:

- For documents, use:  
  `prompt_name="document"`

- For queries, use:  
  `prompt_name="query"`

We also have to always additionally, specify  the `dataset_embeddings`, in order to use the once we produced before.

By doing so, we ensure that the embeddings are generated with the correct context for both retrieval and generation tasks, leveraging the efficient performance of the `cde-small-v1` model.

In [ ]:
doc_embeddings = embeddings_model.encode(
    #[doc.page_content for doc in corpus],
    [doc for doc in corpus],
    prompt_name="document",
    dataset_embeddings=dataset_embeddings, # this is the contexualized embeddings of the minicorpus
    convert_to_tensor=True,
    show_progress_bar=True
)

query_embeddings = embeddings_model.encode(
    [query['question'] for query in queries],
    prompt_name="query",
    dataset_embeddings=dataset_embeddings,  # this is the contexualized embeddings of the minicorpus
    convert_to_tensor=True,
    show_progress_bar=True
)

print(f"Document embeddings shape: {doc_embeddings.shape}")
print(f"Query embeddings shape: {query_embeddings.shape}")

We can now computes similarities between the embeddings and all the queries by simply calling `embeddings_model.similarity` (which uses by default cosine similarity), and inspect some of the results obtained.

In [ ]:
similarities = embeddings_model.similarity(query_embeddings, doc_embeddings)
print("'similarities' shape : ",similarities.shape)
topk_values, topk_indices = similarities.topk(5)

random.seed(4242)
random_queries = random.sample(queries, 2)
for query in random_queries:
    query_idx = queries.index(query)
    print(f"Query: {query['question']}")
    for j, idx in enumerate(topk_indices[query_idx]):
        doc = corpus[idx]
        #print(f"Rank {j+1} (Score: {topk_values[query_idx][j]:.4f}, Doc ID: {idx}): {doc.page_content[:200]}...")
        print(f"Rank {j+1} (Score: {topk_values[query_idx][j]:.4f}, Doc ID: {idx}): {doc[:200]}...")
    print()

## Chroma

While not strictly necessary for this project - since the provided PDF is relatively small - we decided to integrate a vector database to enhance the storage and retrieval of the embeddings produced. This integration becomes particularly valuable when larger files are processed, as they often require splitting the corpus into smaller chunks, with each chunk generating its own embedding. The resulting large number of embeddings makes efficient storage and retrieval mechanisms increasingly important.

After evaluating various options, we selected [**Chroma**](https://www.trychroma.com/) for its simplicity of local deployment and good performances. Chroma is particularly well-suited for managing **large document collections**, providing **faster retrieval** and ensuring that the pipeline remains robust and adaptable to big files.

### Implementation Steps

We begin by creating a Chroma client and verifying if a collection named `'chroma-collection'` already exists locally:  if it does, we delete it to ensure a clean state before initializing a new one.

For reference, a Chroma client can be instantiated in two ways :

- in an **ephemeral way** (in-memory), particularly useful for experimentation, such as testing different embedding functions and retrieval techniques. If data persistence is not a requirement, this approach provides a lightweight and efficient way to get started with Chroma while maintaining flexibility for quick iterations. This can be achieved with the following code:

    ```python
    import chromadb
    client = chromadb.Client()
    ```

- with a **persistent client**. This can be done by providing a  path where Chroma will store its database files on disk, and load them on start.

    ```python
    import chromadb
    client = chromadb.PersistentClient(path="/path/to/save/to")
    ```
    If a `path` is not provided, the default is `.chroma`

More about Chroma can be found in its [documentation](https://docs.trychroma.com/docs/overview/introduction). We choose to utilize the **persistent** approach, so let's follow by instantiating a `PersistentClient`;  if the collection already exists, we delete it to ensure a clean state before initializing a new one.


In [ ]:
from chromadb import PersistentClient, Collection

COLLECTION_NAME = "chroma-collection"

# Initialize the Chroma client
client = PersistentClient(path="./"+COLLECTION_NAME)

# Check if the collection exists and delete it if it does
if COLLECTION_NAME in [col.name for col in client.list_collections()]:
    client.delete_collection(name=COLLECTION_NAME)
    print(f"Collection {COLLECTION_NAME} exists, deleting it")
else:
    print(f"Collection {COLLECTION_NAME} does not exist")

### Integration of `cde-small-v1` with Chroma

Integrating the **`cde-small-v1`** embedding model with Chroma required the creation of a custom `EmbeddingFunction`. This was necessary because **`cde-small-v1`** has a distinct workflow that deviates from standard embedding models. Specifically, as said it uses:

- **`prompt_name`**: This parameter distinguishes between "document" and "query" embeddings, to have proper alignment during retrieval tasks.
- **`dataset_embeddings`**: These provide a reference for conditional embedding generation, improving contextual accuracy in embeddings.

By default, Chroma does not natively support these additional parameters. However, it offers the flexibility of extending the `EmbeddingFunction` protocol, allowing us to integrate the specific requirements of the **`cde-small-v1`** model into Chroma’s workflow.

We can check the type of the elements of the corpus produced by our chunking functions and use it to differentiate between queries and documents.

In [ ]:
print(type(corpus[0]))

In [48]:
from chromadb import Documents, EmbeddingFunction, Embeddings


class CustomEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        """
        Automatically detect if the input is a document or a query and set the mode accordingly.
        :param input: Documents or queries to embed.
        :return: Embeddings.
        """
        # Detect the mode: "document" if the input is a list of documents, else "query".
        mode = "query" #"document" if isinstance(input[0], langchain_core.documents.base.Document) else "query"

        # If the input is a list of Documents, extract the text.
        #if isinstance(input, list):
        #    input = [doc if isinstance(doc, str) else doc.page_content for doc in input]

        # Generate embeddings using the model with the determined mode
        embeddings = embeddings_model.encode(
            input,
            prompt_name=mode,
            dataset_embeddings=dataset_embeddings,
            convert_to_tensor=True,
        ).cpu().numpy()

        return embeddings.tolist()

# Create an instance of the embeddings function for queries (we already calculated all the embeddings of the documents)
custom_embedding_function = CustomEmbeddingFunction()

Furthermore, while Chroma defaults to using Euclidean distance (`l2`) for nearest-neighbor search, the **`cde-small-v1`** embeddings from `SentenceTransformers` are based on cosine similarity by default. To utilize it, We configure the metadata parameter `"hnsw:space": "cosine"` when creating the collection. This ensures that Chroma uses cosine similarity, aligning with the previous behavior.

<style>
    img {
        border-radius: 15px;
    }
</style>
![chroma-similarity-measures.png](https://github.com/giuliocapecchi/LM-project/blob/main/assets/chroma-similarity-measures.png?raw=1)

Once the collection is configured, we add all the documents and their precomputed embeddings to it.

> **NOTE:** We manually computed the embeddings beforehand (for analysis purposes) in the previous steps. If we had only provided the documents, Chroma would have automatically generated the embeddings using the `CustomEmbeddingFunction`, without the need for precomputation.

In [49]:
# Create a Chroma collection with the custom embedding function
collection: Collection = client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=custom_embedding_function,
    get_or_create=True,
    metadata={"hnsw:space": "cosine"} # l2 is the default but cosine is more suitable for this embedding function
)

# Add the documents and their (already computed) embeddings to the collection
collection.add(
    ids=[str(i) for i in range(len(corpus))],  # Unique identifiers for the documents
    embeddings=doc_embeddings.cpu().numpy(),
    #documents=[doc.page_content for doc in corpus],
    documents=[doc for doc in corpus],
)

In [ ]:
#print(collection.peek()) # returns a list of the first 10 items in the collection
print("Number of documents in the corpus: ",len(corpus))
print("Number of documents in the collection: ",collection.count()) # returns the number of items in the collection

We can finally test `collection.query` to make some calls to the Chroma collection, and we expect similar (ideally, the same) results as above, when we used `model.similarity`.

In [ ]:
query_text = "How many bytes can UTF-8 use to encode a character?"

# create a Document object with the query text
results = collection.query(
    query_texts=query_text,
    n_results=5
)
print(results)

# Print the results with the document IDs and scores
for i, (doc_id, score, doc) in enumerate(zip(results['ids'][0], results['distances'][0], results['documents'][0])):
    print(f"Rank {i+1}: Document ID: {doc_id}, Score: {1-score:.4f}, {doc[:200]}")

... which is exactly identical to the previous ones.

One final thing to note is that in Chroma cosine similarity is calculated as `1 - cosim` ; this is done to ensure that lower values indicate a better position in the ranked list. To obtain results consistent with the previous ones, we subtractde the score from one in the above code.


### Compute scores between queries and documents

Now we can analyze a similarity heatmap between the embeddings of our documents and the provided queries. The results show consistency with the following observations:

- **Few documents match the queries**: This suggests that the matching documents are likely the most relevant ones.
- **Matches often occur in subsequent documents**: This is expected since the corpus was split into chunks based on the document's structure. For example, chapters are separated, so answers to specific questions are more likely to be located within the same chapter or adjacent sections of the document.

Additionally, we provide a distribution of the similarity scores. This helps us assess whether an empirical threshold can be established to determine which documents are truly similar to the provided queries.

In [ ]:
from seaborn import heatmap
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot heatmap
heatmap(similarities.cpu().numpy().T, cmap="jet", ax=axes[0])
axes[0].set_title("Similarity Heatmap")

# Plot histogram
axes[1].hist(similarities.cpu().flatten(), bins=50, color='blue', alpha=0.6)
axes[1].set_title("Distribution of Similarity Scores")
axes[1].set_xlabel("Similarity")
axes[1].set_ylabel("Frequency")
axes[1].axvline(x=0.5, color='red', alpha=0.6, linestyle='--')

plt.tight_layout()
plt.show()

## LLM

After conducting some research, we chose to work with one of Meta's latest smaller open-source Llama models available at the time of writing this notebook: [Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct), released the *25th of September 2024*. We specifically selected the *instruct* variant because it is pre-trained to follow basic instructions, offering a more user-friendly and fine-tuned experience compared to the standard non-instruct version (which is also available). Below, we review this model's [specifications](https://llamaimodel.com/requirements-3-2/):

| **Llama 3.2 3B Instruct Model Specifications**                    | **Requirement**       | **Details**                                                                                  |
|----------------------------------|-----------------------|----------------------------------------------------------------------------------------------|
| Parameters                       | 3 billion             |                                                                                              |
| Context Length                   | 128,000 tokens        |                                                                                              |
| **Hardware Requirements**        |                       |                                                                                              |
| CPU and RAM                      |                       | CPU: Multicore processor <br> RAM: Minimum of 16 GB recommended                              |
| GPU                              |                       | NVIDIA RTX series (for optimal performance), at least 8 GB VRAM                              |
| **Estimated GPU Memory Requirements** |                       |                                                                                              |
| Higher Precision Modes           | BF16/FP16             | ~6.5 GB                                                                                      |
| Lower Precision Modes            | FP8                   | ~3.2 GB                                                                                      |
|                                  | INT4                  | ~1.75 GB                                                                                     |
| **Software Requirements**        |                       |                                                                                              |
| Software Dependencies            |                       | Frameworks: PyTorch <br> Libraries: Hugging Face Transformers (version 4.45.0 or higher), CUDA |

In [ ]:
print("Is bf16 supported: ",torch.cuda.is_bf16_supported())

The `bitsandbytes` library is a lightweight Python wrapper around CUDA custom functions, in particular 8-bit optimizers, matrix multiplication (LLM.int8()), and 8 & 4-bit quantization functions.

The library includes quantization primitives for 8-bit & 4-bit operations, through `bitsandbytes.nn.Linear8bitLt` and `bitsandbytes.nn.Linear4bit` and 8-bit optimizers through bitsandbytes.optim module.

#### Quantization Choices

We chose a *middle ground* by loading the model with **4-bit quantization** while maintaining **BFloat16** precision for computation.

The use of 4-bit quantization reduces the precision of the model’s weights to just 4 bits per value, significantly lowering memory usage and accelerating inference. This method retains only the most essential information, sacrificing some numerical precision, but allows for larger models to be handled on GPUs with limited memory. Despite the weights being quantized to 4 bits, the model still performs computations in **16-bit floating point (BFloat16)** precision. BFloat16 is a 16-bit format that preserves much of the dynamic range of floating-point operations while requiring less memory than the traditional 32-bit format. This way, while the 4-bit quantization reduces the memory footprint of the model weights, the computation is performed in BFloat16, optimizing performance on modern GPUs that are tailored for BFloat16 operations. This configuration strikes a balance between computational efficiency and numerical precision, enabling fast inferences with minimal memory usage without significant loss in result quality.

Given the hardware constraints, we opted for **4-bit quantization** using `BitsAndBytes` (as explained later). This approach greatly reduces the memory footprint and speeds up inference, ensuring acceptable performance for our use case. Without this configuration, the GPU’s memory usage was consistently at 100%, and inference times were approximately 2-3 minutes per query. With 4-bit quantization, memory usage drops to about 6GB, preventing GPU overload and reducing inference time to around 30 seconds per query, delivering satisfactory results. Further details on the quantization process will be provided later.

However, we also need to consider that both the embedding model and the LLM must be loaded into memory, which adds another layer of complexity to the memory management. This requires careful balancing, as the total memory usage must accommodate both the LLM and the embedding model simultaneously. We will address how we manage this in the next section.

In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # TODO : riguardare che fa

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"

# Quantization is a technique that reduces the precision of
# the model’s weights to make it run faster and consume less memory,
# often at the cost of a slight reduction in model accuracy or quality
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Lower precision reduces memory usage and can speed up inference (maybe try 8)
    bnb_4bit_use_double_quant=True,         # Using double quantization can help reduce the loss in accuracy associated with quantization
    bnb_4bit_quant_type="nf4",              # Normal Float 4-bit quantization, a scheme that may preserve model quality better than straightforward quantization methods
    bnb_4bit_compute_dtype=torch.bfloat16,  # The internal compute dtype used during inference. bfloat16 (BF16) is often chosen because it’s efficient on modern accelerators
    llm_int8_enable_fp32_cpu_offload=True   # Enable FP32 CPU offload
)


llm_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map={"": device},  # Custom device map to ensure all modules are on GPU
    quantization_config=bnb_config,
).to(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

generation_config = llm_model.generation_config
generation_config.max_new_tokens = 200                    # the maximum number of new tokens the model will generate (long outputs might be more prone to off-topic or repetitive content)
generation_config.min_new_tokens = 1                      # the minimum number of new tokens the model will generate
generation_config.temperature = 0.1                       # it controls the randomness of the generation, lower temp means more deterministic, conservative (less creative) and repetitive answers [about 0.1-1.2]
generation_config.top_p = 0.5                             # nucleus sampling controls how the model picks words based on their cumulative probability, lower value (0.5) means safer, more coherent text but less diverse [about 0.5-0.9]
generation_config.num_return_sequences = 1                # how many separate output sequences are returned for each generation prompt, get multiple different answers in one go, useful for picking the best response from several tries
generation_config.pad_token_id = tokenizer.eos_token_id   # special token IDs that represent padding and the end-of-sequence token. Generally, these are set to ensure the model knows when to stop and how to handle inputs of different lengths
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.repetition_penalty = 2.0                # discourages the model from repeating the same phrases or tokens over and over [about 1.0-2.0] (high value cause the model to avoid some tokens even if they are contextually appropriate)

In [ ]:
# Check GPU usage
gpu_memory_gb = torch.cuda.get_device_properties(device).total_memory / (1024 ** 3)  # in GB
memory_allocated_gb = torch.cuda.memory_allocated(device) / (1024 ** 3)  # in GB
print(f"Memory Allocated: {memory_allocated_gb:.2f} GB")
print(f"Total memory usage: {(memory_allocated_gb / gpu_memory_gb) * 100:.2f}%")

In [ ]:
def retrieve_relevant_documents(query, k=5):
    results = collection.query( # query the Chroma collection
        query_texts=query,
        n_results=k
    )
    return results['documents'][0], results['distances'][0]

# test the function
query = "How many bytes can UTF-8 use to encode a character?"
print(f"Query: {query}")
documents, distances = retrieve_relevant_documents(query)
for doc, distance in (zip(documents, distances)):
    print(f"(score: {1-distance:.4f}) {doc[:200]}")

In [57]:
base_prompt =  '''You are an AI assistant for RAG.
Your task is to provide detailed and accurate answers to user questions based on the provided contexts.
If and only if ALL of the provided context lacks sufficient information, just state: "The provided context does not have the answer." without speculating or providing unsupported answers.
Keep the answers concise and professional and do not add informations that are not asked.

User question: {user_query}

Contexts:
{chunks_information}

Answer:
'''

In [58]:
# Example usage of the RAG system with the PDF
def answer_questions(questions):
    count = 0
    error = 0
    results = {}

    current_time = time.strftime("%m%d-%H%M%S")

    pbar = tqdm(questions, total=len(questions), desc="Answering questions...", unit="question")
    for q in pbar:
        top_k_chunks = retrieve_relevant_documents(q['question'], k=5)
        retrieved_chunks = [chunk for chunk in top_k_chunks]
        prompt = base_prompt.format(user_query=q['question'], chunks_information="\n".join(retrieved_chunks))
        encoding = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.inference_mode():
            outputs = llm_model.generate(
                input_ids=encoding.input_ids,
                attention_mask=encoding.attention_mask,
                generation_config=generation_config,
                num_beams=5,  # Use beam search for better results
                early_stopping=True,  # Stop early if all beams finish
            )

        # Exclude the prompt tokens from the generated output
        generated_tokens = outputs[0][len(encoding.input_ids[0]):]
        generated_unpreprocessed_sequence = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
        match = re.search(r'\b[1-4]\b', generated_unpreprocessed_sequence)
        answer = match.group(0) if match else ""  # first number found or empty string

        with open(f"quiz/runs_basemodel/quiz_answers_{current_time}.txt", "a", encoding="utf-8") as f:
            f.write(f"Question: {q['question']}\nAnswer: {answer}\nCorrect answer:{q['correct']}\nGenerated unpreprocessed sequence: {generated_unpreprocessed_sequence}\n--------------------------------------------------------------------\n\n")

        results[q['question_id']] = answer

        if len(answer) != 1 or answer not in "1234":
            error += 1
        else:  # the format is correct, now check if the answer is correct
            if str(q['correct']) == answer:
                count += 1
        pbar.set_postfix(Corrects=f"{count}/{len(questions)}", Errors=error)

    print("-------------------------\tFINISHED RUN. Error count: ", error, "-------------------------")
    return results, count / len(questions) * 100

In [ ]:
import gradio as gr
from latex2mathml.converter import convert as latex2mathml
import re

def preprocess_latex_formula(text):
    """
    Estrae le formule LaTeX (inline e display) e le converte in MathML,
    rimuovendo parentesi graffe inutili e gestendo simboli complessi.
    """    
    # 1. Escapare le parentesi graffe
    text = text.replace(r'\{', r'\\{').replace(r'\}', r'\\}')
    
    # 2. Gestione delle formule LaTeX in modalità display
    display_formula_pattern = r'(?:\\\[|\$\$)(.*?)(?:\\\]|\$\$)'
    
    # 3. Gestione delle formule LaTeX inline
    inline_formula_pattern = r'\\\((.*?)\\\)|\$(.*?)\$'
    
    def convert_latex_to_mathml(match, mode):
        latex = match.group(1) or match.group(2)
        latex = latex.strip()
        
        try:
            mathml = latex2mathml(latex)
            mathml = mathml.replace("{", "").replace("}", "")
            
            if mode == 'display':
                return f'<div style="text-align: center; margin: 1em 0;">{mathml}</div>'
            else:
                return f'<span style="vertical-align: middle;">{mathml}</span>'
        except Exception as e:
            print(f"Error converting formula: {latex} \nError: {e}")
            return f'[Error converting formula: {latex}]'
    
    text = re.sub(
        display_formula_pattern,
        lambda match: convert_latex_to_mathml(match, 'display'),
        text,
        flags=re.DOTALL
    )
    
    text = re.sub(
        inline_formula_pattern,
        lambda match: convert_latex_to_mathml(match, 'inline'),
        text,
        flags=re.DOTALL
    )
    
    return text

def query_rag_model(user_query):
    # Recupero dei documenti
    documents, scores = retrieve_relevant_documents(user_query, k=5)
   
    print("documents and their scores: ")
    for doc, score in zip(documents, scores):
        print(f"Score: {1-score:.4f}, {doc[:200]}")
    
    # Prompt per il modello
    prompt = base_prompt.format(user_query=user_query, chunks_information="\n".join(documents))
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.inference_mode():
        outputs = llm_model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
            num_beams=5,
            early_stopping=True,
        )
    
    generated_tokens = outputs[0][len(encoding.input_ids[0]):]
    generated_unpreprocessed_sequence = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    print("Generated Text:", generated_unpreprocessed_sequence,"\n")
    
    # Converti le formule LaTeX in MathML
    processed_output = preprocess_latex_formula(generated_unpreprocessed_sequence)
    return processed_output

# Creazione dell'interfaccia Gradio con stile consistente
with gr.Blocks(css="""
    .input-box { border: 1px solid #ccc; border-radius: 4px; padding: 10px; margin: 10px 0; }
    .output-box { border: 1px solid #ccc; border-radius: 4px; padding: 10px; margin: 10px 0; background-color: #373535; height: auto; }

""") as demo:
    gr.Markdown("# RAG Model Query Interface")
    gr.Markdown("Ask questions to the RAG model and get answers based on the provided PDF context.")
    
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Enter your query",
                placeholder="Type your question here...",
                elem_classes=["input-box"]
            )
    
    with gr.Row():
        with gr.Column():
            output_text = gr.HTML(
                label="Answer",
                elem_classes=["output-box"]
            )
    
    input_text.submit(
        fn=query_rag_model,
        inputs=input_text,
        outputs=output_text
    )

# Avvia l'interfaccia

demo.launch()